<a href="https://colab.research.google.com/github/a-essa/Sentiment-Analysis-and-Satisfaction-Prediction/blob/master/ProjetTripAdvisor_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import os
import io
from tensorflow import keras
from tensorflow.keras import layers
import keras.preprocessing.text
import tensorflow.keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import multilabel_confusion_matrix as mcm
import tensorflow_hub as hub
tf.keras.backend.clear_session()
np.set_printoptions(precision=3, suppress=True)
pd.options.display.max_colwidth = 1000
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
print(tf.__version__)

2.0.0


In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

# Read CSV

In [0]:
Total_data_for_tokenizer=pd.read_csv('/content/gdrive/My Drive/Data/Total_data_for_tokenizer.csv' , sep='\t')

In [0]:
training_set_TripAdvisor=pd.read_csv('/content/gdrive/My Drive/Data/training_set_TripAdvisor.csv' , sep='\t')

In [0]:
test_set_TripAdvisor=pd.read_csv('/content/gdrive/My Drive/Data/test_set_TripAdvisor.csv' , sep='\t')

In [0]:
training_set_IMDB=pd.read_csv('/content/gdrive/My Drive/Data/training_set_IMDB.csv' , sep='\t')

In [0]:
test_set_IMDB=pd.read_csv('/content/gdrive/My Drive/Data/test_set_IMDB.csv' , sep='\t')

In [0]:
del Total_data_for_tokenizer['Unnamed: 0']
del training_set_TripAdvisor['Unnamed: 0']
del test_set_TripAdvisor['Unnamed: 0']
del training_set_IMDB['Unnamed: 0']
del test_set_IMDB['Unnamed: 0']

# Tokenizer

In [0]:
def create_dataset(dataframe):
  msk = np.random.rand(len(dataframe)) < 0.8

  train = dataframe[msk]

  test = dataframe[~msk]
  #print(train_file_path.head)
  all_data = np.array(dataframe.values.tolist())
  training_set = np.array(train.values.tolist())
  test_set=np.array(test.values.tolist())
  print("Dataset Length: ",len(training_set)+ len(test_set))
  return all_data, training_set, test_set

In [0]:
def create_tokenizer(dataset):
  tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token='UNK')
  # fit the tokenizer on the documents
  tokenizer.fit_on_texts(dataset[:,1])
  # summarize what was learned
  print("Word counter: ",tokenizer.word_counts)
  print("Number of sentences: ",tokenizer.document_count)
  print("Word Index: ", tokenizer.word_index)
  print("Word Docs: ",tokenizer.word_docs)
  return tokenizer

In [0]:
def vocab_size(tokenizer):
  voca_size=len(tokenizer.word_counts)+2   # 1 ?
  return voca_size

In [0]:
def maxlen(data):
  return max([len(x) for x in data])

In [0]:
Total_data_for_tokenizer_np = np.array(Total_data_for_tokenizer.values.tolist())

In [0]:
tokenizer=create_tokenizer(Total_data_for_tokenizer_np)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
def tokenize_sentences(input_set):
  input_sequences = []
  for line in input_set[:,1]:
      token_list = tokenizer.texts_to_sequences([line])[0]
      input_sequences.append(token_list)
  return input_sequences

In [0]:
training_set_IMDB_np = np.array(training_set_IMDB.values.tolist())
test_set_IMDB_np = np.array(test_set_IMDB.values.tolist())
training_set_TripAdvisor_np = np.array(training_set_TripAdvisor.values.tolist())
test_set_TripAdvisor_np = np.array(test_set_TripAdvisor.values.tolist())

In [0]:
input_training_set_IMDB = tokenize_sentences(training_set_IMDB_np)
input_test_set_IMDB = tokenize_sentences(test_set_IMDB_np)
input_training_set_TripAdvisor = tokenize_sentences(training_set_TripAdvisor_np)
input_test_set_TripAdvisor = tokenize_sentences(test_set_TripAdvisor_np)

In [0]:
def pad_sentences(input_sequences, max_sequence_len):
  input_padded_sequences = np.array(pad_sequences(input_sequences,   
                            maxlen=max_sequence_len, padding='pre'))
  return input_padded_sequences

In [0]:
def pad_sentences_post(input_sequences, max_sequence_len):
  input_padded_sequences = np.array(pad_sequences(input_sequences,   
                            maxlen=max_sequence_len, padding='post'))
  return input_padded_sequences

In [0]:
def maxlen(data1, data2 , data3 , data4):
  max_len_data1 = max([len(x) for x in data1])
  max_len_data2 = max([len(x) for x in data2])
  max_len_data3 = max([len(x) for x in data3])
  max_len_data4 = max([len(x) for x in data4])
  return max(max_len_data1, max_len_data2 , max_len_data3 , max_len_data4)

In [0]:
max_len = maxlen(input_training_set_IMDB, input_test_set_IMDB, input_training_set_TripAdvisor, input_test_set_TripAdvisor)

#Embeddings pretraining (Next Word Prediction)


In [0]:
def split_sentences_forward(input_set):
  input_sequences = []
  for line in input_set:
      for i in range(1, len(line)):
        if i < 99 :
          n_gram_sequence = line[:i+1]
        else :
          n_gram_sequence = line[i-99:i+1]
        input_sequences.append(n_gram_sequence)
  return input_sequences

In [0]:
def split_sentences_Backward(input_set):
  input_sequences = []
  for line in input_set[:,1]:
      token_list = tokenizer.texts_to_sequences([line])[0]
      for i in range(1, len(token_list)):
        if i < 99 :
          n_gram_sequence = token_list[:i+1]
        else :
          n_gram_sequence = token_list[i-99:i+1]
        input_sequences.append(n_gram_sequence)
  return input_sequences

In [0]:
def create_dataset_slices_embaddings(input_sequences):
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  #label = tf.keras.utils.to_categorical(label, num_classes=voc_size)
  return predictors, label

In [0]:
def create_preprocessed_dataset(predictors, labels):
  dataset = tf.data.Dataset.from_tensor_slices((predictors, labels))
  return dataset

In [0]:
def top_k_categorical_accuracy1(y_true, y_pred, k=10):
    return K.mean(K.in_top_k(K.cast(y_pred,dtype='float32'),K.argmax(y_true, axis=-1), k), axis=-1)

In [0]:
input_train_WE_1=split_sentences_forward(input_training_set_IMDB)

In [0]:
input_train_WE_2=split_sentences_forward(input_training_set_TripAdvisor)

In [0]:
input_test_WE_1=split_sentences_forward(input_test_set_IMDB[5000:])

In [0]:
input_test_WE_2=split_sentences_forward(input_test_set_TripAdvisor)

In [0]:
input_train_WE_sequences_1 = pad_sentences(input_train_WE_1, 100)
input_train_WE_sequences_2 = pad_sentences(input_train_WE_2, 100)
input_test_WE_sequences_1 = pad_sentences(input_test_WE_1, 100)
input_test_WE_sequences_2 = pad_sentences(input_test_WE_2, 100)

In [0]:
training_predictors_WE, training_label_WE=create_dataset_slices_embaddings(input_train_WE_sequences_1)
test_predictors_WE, test_label_WE= create_dataset_slices_embaddings(input_test_WE_sequences_1)

In [0]:
training_predictors_WE_2, training_label_WE_2=create_dataset_slices_embaddings(input_train_WE_sequences_2)
test_predictors_WE_2, test_label_WE_2= create_dataset_slices_embaddings(input_test_WE_sequences_2)

In [0]:
voc_size = vocab_size(tokenizer)

In [0]:
EmbeddingLayer = tf.keras.layers.Embedding(voc_size, 64)
LstmLayer = tf.keras.layers.LSTM(150)
DenseLayerEmbedding = tf.keras.layers.Dense(64, activation='relu')
DropLayerEmbedding = tf.keras.layers.Dropout(0.1)
DenseLayerOutputEmbedding = tf.keras.layers.Dense(voc_size, activation='softmax')

In [0]:
model = tf.keras.Sequential()
model.add(EmbeddingLayer)
model.add(LstmLayer)
model.add(DenseLayerEmbedding)
model.add(DropLayerEmbedding)
model.add(DenseLayerOutputEmbedding)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5120512   
_________________________________________________________________
lstm (LSTM)                  (None, 150)               129000    
_________________________________________________________________
dense (Dense)                (None, 64)                9664      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 80008)             5200520   
Total params: 10,459,696
Trainable params: 10,459,696
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [0]:
checkpoint_path = "training_Embadding/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

In [0]:
history = model.fit(training_predictors_WE, 
                    training_label_WE,
                    epochs=2,
                    batch_size=100,
                    validation_data=(test_predictors_WE, test_label_WE),
                    verbose=1,
                    callbacks=[cp_callback])

Train on 3616784 samples, validate on 2831359 samples
Epoch 1/2
3616700/3616784 [============================>.] - ETA: 0s - loss: 7.6951 - accuracy: 0.0596
Epoch 00001: saving model to training_Embadding/cp-0001.ckpt
3616784/3616784 [==============================] - 2552s 706us/sample - loss: 7.6951 - accuracy: 0.0596 - val_loss: 7.5867 - val_accuracy: 0.0709
Epoch 2/2
2262000/3616784 [=================>............] - ETA: 12:57 - loss: 7.4362 - accuracy: 0.0723Buffered data was truncated after reaching the output size limit.

In [0]:
history = model.fit(training_predictors_WE_2, 
                    training_label_WE_2,
                    epochs=2,
                    batch_size=100,
                    validation_data=(test_predictors_WE_2, test_label_WE_2),
                    verbose=1,
                    callbacks=[cp_callback])

Train on 947983 samples, validate on 237895 samples
Epoch 1/2
947900/947983 [============================>.] - ETA: 0s - loss: 5.7982 - accuracy: 0.1446
Epoch 00001: saving model to training_Embadding/cp-0001.ckpt
947983/947983 [==============================] - 547s 577us/sample - loss: 5.7982 - accuracy: 0.1446 - val_loss: 5.1032 - val_accuracy: 0.1927
Epoch 2/2
947900/947983 [============================>.] - ETA: 0s - loss: 4.9881 - accuracy: 0.1962
Epoch 00002: saving model to training_Embadding/cp-0002.ckpt
947983/947983 [==============================] - 528s 557us/sample - loss: 4.9881 - accuracy: 0.1962 - val_loss: 4.7930 - val_accuracy: 0.2161


#Dataset slices

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
input_train_CL_sequences_1 = pad_sentences_post(input_training_set_IMDB, max_len)
input_train_CL_sequences_2 = pad_sentences_post(input_training_set_TripAdvisor, max_len)
input_test_CL_sequences_1 = pad_sentences_post(input_test_set_IMDB, max_len)
input_test_CL_sequences_2 = pad_sentences_post(input_test_set_TripAdvisor, max_len)

In [0]:
def create_dataset_slices_sentence_Regre(input_sequences, targets):
  predictors= tf.constant(input_sequences)
  targets=targets.astype(int)
  return predictors, targets

In [0]:
training_predictors, training_label=create_dataset_slices_sentence_Regre(input_train_CL_sequences_2,training_set_TripAdvisor_np[:,0])
test_predictors, test_label=create_dataset_slices_sentence_Regre(input_test_CL_sequences_2,test_set_TripAdvisor_np[:,0])
training_predictors_tfds, training_label_tfds=create_dataset_slices_sentence_Regre(input_train_CL_sequences_1,training_set_IMDB_np[:,0])
test_predictors_tfds, test_label_tfds=create_dataset_slices_sentence_Regre(input_test_CL_sequences_1,test_set_IMDB_np[:,0])

In [0]:
from keras.regularizers import l2

#Model

In [0]:
BidirectionalLayer1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))
BidirectionalLayer2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))
GlobalMaxPool1DLayer = tf.keras.layers.GlobalMaxPool1D()
DenseLayer1 = tf.keras.layers.Dense(40, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) , activation='relu')
DropLayer = tf.keras.layers.Dropout(0.1)
DenseLayerOutputRE = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear)

In [0]:
EmbeddingLayer.trainable = False

In [0]:
modelBin = tf.keras.Sequential()
modelBin.add(EmbeddingLayer)
modelBin.add(BidirectionalLayer1)
modelBin.add(BidirectionalLayer2)
modelBin.add(GlobalMaxPool1DLayer)
modelBin.add(DenseLayer1)
modelBin.add(DropLayer)
modelBin.add(DenseLayerOutputRE)
modelBin.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5120512   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         66048     
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 128)         98816     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                5160      
_________________________________________________________________
dropout_1 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [0]:
modelBin.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae'])

In [0]:
history = modelBin.fit(training_predictors_tfds, 
                    training_label_tfds,
                    epochs=3,
                    batch_size=100,
                    validation_data=(test_predictors_tfds, test_label_tfds),
                    verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 158s 6ms/sample - loss: 0.3988 - mse: 0.1875 - mae: 0.3710 - val_loss: 0.2184 - val_mse: 0.1716 - val_mae: 0.3166
Epoch 2/3
25000/25000 [==============================] - 152s 6ms/sample - loss: 0.1737 - mse: 0.1491 - mae: 0.3064 - val_loss: 0.1576 - val_mse: 0.1442 - val_mae: 0.3057
Epoch 3/3
24300/25000 [============================>.] - ETA: 2s - loss: 0.1416 - mse: 0.1319 - mae: 0.2760

In [0]:
EmbeddingLayer.trainable = True 

In [0]:
modelBin.compile(loss='mean_squared_error', 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [0]:
history = modelBin.fit(training_predictors_tfds, 
                    training_label_tfds,
                    epochs=2,
                    batch_size=100,
                    validation_data=(test_predictors_tfds, test_label_tfds),
                    verbose=1)

In [0]:
model1 = tf.keras.Sequential()
model1.add(EmbeddingLayer)
model1.add(BidirectionalLayer1)
model1.add(BidirectionalLayer2)
model1.add(GlobalMaxPool1DLayer)
model1.add(DenseLayer1)
model1.add(DropLayer)
model1.add(DenseLayerOutputRE)

model1.summary()

In [0]:
EmbeddingLayer.trainable = False
BidirectionalLayer1.trainable = False
BidirectionalLayer2.trainable = False
GlobalMaxPool1DLayer.trainable = False
DenseLayer1.trainable = False

In [0]:
model1.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [0]:
history = model1.fit(training_predictors, 
                    training_label,
                    epochs=5,
                    batch_size=100,
                    validation_data=(test_predictors, test_label),
                    verbose=1)

In [0]:
DenseLayer1.trainable = True

In [0]:
model1.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [0]:
history = model1.fit(training_predictors, 
                    training_label,
                    epochs=4,
                    batch_size=100,
                    validation_data=(test_predictors, test_label),
                    verbose=1)

In [0]:
EmbeddingLayer.trainable = True
BidirectionalLayer1.trainable = True
BidirectionalLayer2.trainable = True
GlobalMaxPool1DLayer.trainable = True

In [0]:
model1.compile(loss='mean_squared_error',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['mse','mae','accuracy'])

In [0]:
history = model1.fit(training_predictors, 
                    training_label,
                    epochs=4,
                    batch_size=100,
                    validation_data=(test_predictors, test_label),
                    verbose=1)

#From saved weights

In [0]:
'''EmbeddingLayer = tf.keras.layers.Embedding(80008, 64)
BidirectionalLayer1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))
BidirectionalLayer2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))
GlobalMaxPool1DLayer = tf.keras.layers.GlobalMaxPool1D()
DenseLayer1 = tf.keras.layers.Dense(40, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) , activation='relu')
DropLayer = tf.keras.layers.Dropout(0.1)
DenseLayerOutputRE = tf.keras.layers.Dense(1, activation=tf.keras.activations.linear)'''

In [0]:
'''model1 = tf.keras.Sequential()
model1.add(EmbeddingLayer)
model1.add(BidirectionalLayer1)
model1.add(BidirectionalLayer2)
model1.add(GlobalMaxPool1DLayer)
model1.add(DenseLayer1)
model1.add(DropLayer)
model1.add(DenseLayerOutputRE)

model1.summary()'''

In [0]:
'''model1.load_weights('/content/gdrive/My Drive/Chekpoints/model_checkpoint')'''

#Validation

In [0]:
from sklearn.metrics import mean_squared_error

In [0]:
import nltk

nltk.download("stopwords")
nltk.download("punkt")

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
  
def clean_set(set_len):
  stop_words = list(stopwords.words('english'))
  stop_words.remove("not")
  stop_words.remove("no")
  stop_words.remove("isn't")
  stop_words.remove("hasn't")
  stop_words.remove("wasn't")
  stop_words.remove("didn't")
  stop_words.remove("haven't")
  stop_words.remove("don't")
  stop_words.remove("doesn't")
  stop_words.remove("weren't")
  stop_words.remove("shouldn't")
  lemmatizer = WordNetLemmatizer() 
  for elem in set_len:
    word_tokens = word_tokenize(elem[1]) 
    filtered_sentence = [] 
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(lemmatizer.lemmatize(w)) 
    elem[1]=' '.join(filtered_sentence)

In [0]:
column_names = ['reviews.rating','reviews.text']
eval_data = pd.read_csv('http://christophe-rodrigues.fr/eval_reviews.csv', usecols=column_names, sep=";")

In [0]:
eval_data = np.array(eval_data.values.tolist())

In [0]:
clean_set(eval_data)

In [0]:
eval_data_Text = tokenize_sentences(eval_data)

In [0]:
input_eval = pad_sentences_post(eval_data_Text, max_len)

In [0]:
eval_predictors, eval_label =create_dataset_slices_sentence_Regre(input_eval,eval_data[:,0])

In [0]:
prediction = model1.predict(eval_predictors)

In [0]:
mse = mean_squared_error(prediction, eval_label)

In [0]:
mse